In [1]:
import torch
import os
from dataset import CDEDataset
from torchvision import transforms
from torch.utils.data import DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available(): 
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [2]:
modeltest = torch.load("sweeps/2e-05-8-adam.pt")

In [3]:
root_ims = '../CDE_Data/train/images'
root_ann = '../CDE_Data/train/density_gt'
im_list = os.listdir(root_ims)

train = CDEDataset(im_list,root_ims,root_ann, transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]))

root_ims = '../CDE_Data/val/images'
root_ann = '../CDE_Data/val/density_gt'
im_list = os.listdir(root_ims)

val = CDEDataset(im_list,root_ims,root_ann, transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train = False)




In [4]:
root_ims = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_A/test_data/images'
root_ann = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_A/test_data/density_gt'
im_list = os.listdir(root_ims)
test_SH_A = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

root_ims = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_B/test_data/images'
root_ann = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_B/test_data/density_gt'
im_list = os.listdir(root_ims)
test_SH_B = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

root_ims = '../CDE_Data/UCF_CC_50/images'
root_ann = '../CDE_Data/UCF_CC_50/density_gt'
im_list = os.listdir(root_ims)
test_UCF = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

In [5]:
test_SH_A_dataloader = DataLoader(test_SH_A, batch_size=1, shuffle=True)
test_SH_B_dataloader = DataLoader(test_SH_B, batch_size=1, shuffle=True)
test_UCF_dataloader = DataLoader(test_UCF, batch_size=1, shuffle=True)

test_dataloaders = [test_SH_A_dataloader, test_SH_B_dataloader, test_UCF_dataloader]

In [6]:
def evaluate(model,dataloaders):

    maes = []
    mses = []
    for loader in dataloaders:
        losses =[]

        model.eval()
        with torch.no_grad():    
            for i, data in enumerate(loader, 0):
                # evaluate the model on the test set
                inputs, labels, true_count = data


                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                pred_count = np.sum(outputs[0][0].cpu().detach().numpy())
                true_count = true_count.cpu().detach().numpy()

                losses.append(pred_count - true_count)

                print(i, end = " ")
        mae = np.mean(np.abs(losses))
        mse = np.sqrt(np.mean(np.square(losses)))
        maes.append(mae)
        mses.append(mse)
    return maes, mses
        

In [7]:
maes, mses = evaluate(modeltest, test_dataloaders)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 

In [9]:
maes

[95.36264857617053, 25.23877566373801, 460.76585258483885]

In [10]:
mses

[160.31460594080835, 39.49576838816053, 687.8697389422156]

In [11]:
ex = test_UCF[5]
im = ex[0].unsqueeze(0).to(device)
im.shape

start = time.time()
a = modeltest(im)

print("inference time:", time.time() - start)
print("actual: ",ex[2])
print("predicted: ",np.sum(a[0][0].cpu().detach().numpy()))

fig, axs = plt.subplots(1, 3, figsize=(15,20))
axs[0].imshow(ex[0][0].detach().numpy())
axs[1].imshow(ex[1][0].detach().numpy())
axs[2].imshow(a[0][0].cpu().detach().numpy())
plt.show()

RuntimeError: CUDA out of memory. Tried to allocate 44.00 MiB (GPU 0; 4.00 GiB total capacity; 2.76 GiB already allocated; 10.73 MiB free; 2.79 GiB reserved in total by PyTorch)